# Chat with your data

### This file is for educational purpose, it explains the flow from data to chat.

### The easiest way to run this file is in Google Colab.
### https://colab.research.google.com/
### Log in with your Google-account and add this file in "Ladda Upp".

### In Colab you can run all cells with Ctrl+F9 or press Shift+Enter to run chosen cell.


In [ ]:
# In this cell we install our packages

!pip install -qU \
    langchain \
    openai==0.28.0 \
    pinecone-client==2.2.4 \
    tiktoken==0.5.1

In [ ]:

import os


# Add your unique key here.
os.environ["OPENAI_API_KEY"] ="ENTER-YOUR-OPENAI-KEY-HERE"

### If you have the data in a excel-file you can use the read_csv function. We have removed the top line of the excel-file and and saved it as a csv before we insert the the path in the function.
### All the columns are used as the data. We choose to do this so it would be more easier to use any machine data. All the missing values will be representated by 'null'. This is not necessary but it will help to ai if the data is consistent.

In [ ]:


import pandas as pd
import numpy as np

# If you are running this in Google Colab you have to add the file to a folder. Press the folder icon on the left side of screen. Add the path to your file below.
data = pd.read_csv(r"data\2830612_VMMS_EWO_KPI_clean.csv", sep=";")


data.replace({np.nan: 'null', 'NA': 'null', 'N/A': 'null', 'Missing': 'null'}, inplace=True)

In [ ]:
# Take a look at the data. This code only shows the first five rows.
data.head()

In [ ]:
# We have custom made our own Documentloader.

from typing import Any, Iterator, List

import pandas as pd

from langchain.docstore.document import Document
from langchain.document_loaders.base import BaseLoader


class DataFrameLoader2(BaseLoader):
    """Load from a Pandas DataFrame."""

    def __init__(self, data_frame: Any, ewo_wo_no_column: str = "EWO WO No"):
        """Initialize with a Pandas DataFrame.

        Args:
            data_frame: Pandas DataFrame.
            ewo_wo_no_column: Name of the column to include as metadata. Defaults to "EWO WO No".
        """
        if not isinstance(data_frame, pd.DataFrame):
            raise ValueError(
                f"Expected data_frame to be a pd.DataFrame, got {type(data_frame)}"
            )
        self.data_frame = data_frame
        self.ewo_wo_no_column = ewo_wo_no_column

    def lazy_load(self) -> Iterator[Document]:
        """Lazy load records from the Pandas DataFrame."""

        for _, row in self.data_frame.iterrows():
            text = "\n".join([f"{key}: {value}" for key, value in row.items()])
            metadata = row.to_dict()
            ewo_wo_no_value = row[self.ewo_wo_no_column]
            metadata["EWO WO No"] = ewo_wo_no_value
            yield Document(page_content=text, metadata=metadata, column_names=list(self.data_frame.columns))

    def load(self) -> List[Document]:
        """Load full Pandas DataFrame."""
        return list(self.lazy_load())

### There ara many types of loaders that create the data into a Document. A Document consist of page_content and metadata
### In the different types of loaders you can specify which data/column is page_content and which is metadata.

### Read all about it here https://python.langchain.com/docs/modules/data_connection/document_loaders

In [ ]:

loader = DataFrameLoader2(data)

data = loader.load()

In [ ]:
data[0]

# Show the first Document

In [ ]:
data[0].metadata

# Run this cell to show the metadata

In [ ]:
# Instead of the csv you can read in the data as a json-file. Add the file to your folder and change the file path in the code.

json = pd.read_json(r"C:\Users\debbi\NLP\vol.json", orient=True)

loader = DataFrameLoader2(json)
data = loader.load()


### Embeddings and storing data

In [ ]:
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings


embeddings = OpenAIEmbeddings()


In [ ]:
# Here we assign the FAISS vector index to be the chats database. 
# FAISS (Facebook AI simularity search) is Meta library for handeling similarity searches..
db = FAISS.from_documents(data, embeddings)

In [ ]:
# You can query your database.

query = db.similarity_search("Våg 1")


In [ ]:
# The top most simulary documents to your query
query

### The model 

In [ ]:
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(
    openai_api_key=os.environ["OPENAI_API_KEY"],
    model='gpt-4'
)


In [ ]:
chat

### Prompt

In [ ]:
# You can change the prompt but the {question} and {context} is key values that the models needs. Removing these will cause an error.

from langchain.prompts import PromptTemplate

template = """I will provide you with historical data regarding previous work orders (EWO) for a machine. By examining the historical data, you should respond by describing similar issues in the historical data and how those issues were resolved. Use all the columns and explain how a similar problem was previously resolved.

Before you do that, ask for a brief description of the issue that we will analyze using the historical data you've received.

This description may be concise and may not exactly match any previous ones in the history. Nevertheless, you should search the history for similar problems or solutions resembling the new issue description.

Especially, check what's mentioned in EWO WO Directive, EWO WO Work Done, and EWO WO Work Details. Also, think broadly because there might be misspellings or similar words with the same meaning.

Reply with the number of work orders available. Also, mention how many of them are similar to the issue we're currently analyzing.

Then, proceed with details about all of them. This includes all analyses you can perform about what was done, the time taken, parts replaced, downtime, etc. Time analyses are crucial. Specify the time taken to resolve the issue, the duration from discovery to operation, and, most importantly, thoroughly analyze the root cause. Include all time-related information.

If you encounter any issues while reading the file, do not mention it, just continue. Do not describe all the steps you take. The information you are getting is EWO and you will analyzie it with as much information as needed. The EWO WO ID is the link between the texts. If I ask you for date or time, this is the correct format to look for: YYYY-MM-DD HH:MI:SS. Give the answer about the date if someone asks. If you don't know the answer, just say that you don't know, don't try to make up an answer. If you encounter  some technical terms may not have direct translations in English write the whole answer in english.
If you are looking for a datetime the format to look for is YYYY:MM:DD HH:MI:SS. Don't answer if the chat_history is empty.
{context}


Question: {question}

Answer in Swedish:"""


PROMPT = PromptTemplate(
    template=template, input_variables=["question", "context"]
)

### If you want the chat to remember the conversation you need a memory.

In [ ]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key="chat_history")

In [ ]:
memory

### Time for the final step - putting all the pieces together

In [ ]:
from langchain.chains import RetrievalQA

gpt_pipeline = RetrievalQA.from_chain_type(
    llm=chat, chain_type='stuff',
    chain_type_kwargs = {"prompt": PROMPT},
    memory = memory,
    retriever=db.as_retriever()
)

In [ ]:
gpt_pipeline("Hej")

In [ ]:

# If you print the pipeline you can see all the information about the object.
print(gpt_pipeline)

In [ ]:
gpt_pipeline("Kan du hjälpa mig hitta ett datum för när vi senast hade problem med sandvagnar")

In [ ]:
gpt_pipeline("Kan du hämta Kaizen åtgärd för detta?")

In [ ]:

gpt_pipeline("Har detta hänt tidigare? Hur löstes det då?")

In [ ]:
gpt_pipeline("Hur långt tid tog det att lösa problem med Våg 1 vid varje fall?")

In [ ]:
gpt_pipeline("Finns det en Kaizen åtgärds gjort på fall rörande våg 1?")


In [ ]:
gpt_pipeline("vilken tidpunkt löstes det?")